Output parsers in LangChain transform raw LLM text output into structured, usable formats. They're essential because LLMs primarily generate free-form text, but applications need consistent, machine-readable data (like JSON, lists, or Pydantic objects).

`with_structured_output()` is a convenience for models with **native structured output capabilities** (like OpenAI's newer models via JSON mode or function calling). These models are designed to directly produce schema-compliant JSON, making extraction highly reliable.

For **open-source or older LLMs that lack native structured output**, you *must* use explicit output parsers. You prompt the LLM to output in a specific format (e.g., JSON), and then the parser attempts to extract and validate that format from the raw text response. This is generally less robust and requires more careful prompt engineering.

-----

### Common Output Parsers and Their Usage ⚙️

Here are some key LangChain output parsers:

#### `StrOutputParser`

  * **Purpose**: The simplest parser; it extracts the top-most likely string output from an LLM's `Generation` object. Essentially, it just gets the raw text response.
  * **Usage**: Useful when you only need the plain text output and no specific structured formatting is required.
    ```python
    from langchain_core.output_parsers import StrOutputParser

    # Assuming 'model' is your LLM and 'prompt' is your PromptTemplate
    chain = prompt | model | StrOutputParser()
    result = chain.invoke({"query": "Hello, how are you?"}) # result will be a string
    ```

#### `JsonOutputParser`

  * **Purpose**: Parses LLM output that is expected to be a **JSON string** into a Python dictionary. It's often used with a prompt that explicitly tells the LLM to output JSON.
  * **Usage**: When you need arbitrary JSON, or if you provide a JSON Schema in your prompt and expect the LLM to adhere to it (though not strictly guaranteed by the parser alone for all models).
    ```python
    from langchain_core.output_parsers import JsonOutputParser
    from langchain_core.prompts import PromptTemplate

    parser = JsonOutputParser()

    # Prompt instructs the LLM to output JSON
    prompt = PromptTemplate(
        template="Answer the user query.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    # Assuming 'model' is your LLM
    chain = prompt | model | parser
    result = chain.invoke({"query": "Tell me about a person named Alice"}) # result will be a dict
    ```

#### `PydanticOutputParser`

  * **Purpose**: The most robust way to get structured output from LLMs when not using native `with_structured_output` (or as a component within it). It attempts to parse the LLM's text output directly into an instance of a **Pydantic `BaseModel`**, providing strong **validation** and type safety. It also generates specific formatting instructions for the LLM to follow.
  * **Usage**: Ideal when you have a predefined schema for your output and need strict validation. Crucial for ensuring the data types and structure are correct.
    ```python
    from langchain_core.output_parsers import PydanticOutputParser
    from langchain_core.prompts import PromptTemplate
    from pydantic import BaseModel, Field
    from typing import Literal

    class SentimentAnalysis(BaseModel):
        sentiment: Literal["positive", "negative", "neutral"] = Field(description="Overall sentiment of the text")
        score: float = Field(description="Sentiment score between 0.0 and 1.0")

    parser = PydanticOutputParser(pydantic_object=SentimentAnalysis)

    prompt = PromptTemplate(
        template="Analyze the sentiment of the following text.\n{format_instructions}\nText: {text}\n",
        input_variables=["text"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Assuming 'model' is your LLM
    chain = prompt | model | parser
    result = chain.invoke({"text": "This movie was absolutely fantastic!"}) # result will be a SentimentAnalysis object
    ```